In [ ]:
import pandas as pd
import numpy as np  

In [ ]:
import pandas as pd

# Define o número de linhas a serem puladas
linhas_para_pular = 3  # Defina o número de linhas a serem puladas, incluindo o cabeçalho

# Lê o arquivo CSV pulando as linhas iniciais
df = pd.read_csv('202310COOPERATIVAS.CSV', delimiter=';', skiprows=linhas_para_pular, encoding='ISO-8859-1')

# Exibe as primeiras linhas do DataFrame
df.head()

In [ ]:
# Filtra as linhas onde o valor da coluna "NOME_INSTITUICAO" contém a string "Sicredi" (ignorando maiúsculas e minúsculas) e remove duplicatas
cooperativas_sicredi = df[df['NOME_INSTITUICAO'].str.contains('Sicredi', case=False)].drop_duplicates(subset=['NOME_INSTITUICAO'])

cooperativas_sicredi.head()


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Defina o número de linhas a serem puladas, incluindo o cabeçalho
linhas_para_pular = 3

# Lê o arquivo CSV pulando as linhas iniciais
df = pd.read_csv('202310COOPERATIVAS.CSV', delimiter=';', skiprows=linhas_para_pular, encoding='ISO-8859-1')

# Filtra as linhas onde o valor da coluna "NOME_INSTITUICAO" contém a string "Sicredi" (ignorando maiúsculas e minúsculas) e remove duplicatas
cooperativas_sicredi = df[df['NOME_INSTITUICAO'].str.contains('Sicredi', case=False)].drop_duplicates(subset=['NOME_INSTITUICAO'])

# Configurações do perfil do Chrome para aceitar cookies automaticamente e abrir uma janela anônima
chrome_options = Options()
chrome_options.add_argument("--enable-automation")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--incognito")  # Abrir janela anônima

# Habilitar o gerenciamento de cookies no perfil
chrome_options.add_argument("--enable-blink-features=AutomaticCookieManagementEnabled")

# Caminho para o perfil do Chrome (você pode alterar este caminho conforme necessário)
chrome_options.add_argument("--user-data-dir=/path/to/your/chrome/profile")

# Inicializar o driver do Chrome com as configurações do perfil
driver = webdriver.Chrome(options=chrome_options)

# Defina o número de segundos de atraso entre cada busca
atraso_entre_buscas = 3  # Altere conforme necessário

try:
    urls = []  # Lista para armazenar as URLs encontradas

    # Iterar sobre as informações da coluna NOME_INSTITUICAO de cooperativas_sicredi
    for index, row in cooperativas_sicredi.iterrows():
        # Abrir uma nova janela anônima para cada iteração
        driver.execute_script("window.open('about:blank', 'tab" + str(index) + "');")
        driver.switch_to.window("tab" + str(index))

        # Abrir a página de busca
        driver.get("https://www.sicredi.com.br/site/busca-resultado/")

        # Obter o valor da coluna "NOME_INSTITUICAO" atual
        search_query = row['NOME_INSTITUICAO']

        # Esperar até que o campo de busca esteja disponível
        search_box = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".blocoFormulario input.inputGeral"))
        )

        # Limpar o campo de busca
        search_box.clear()

        # Preencher o campo de busca com o valor atual da coluna
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)

        # Esperar até que o contêiner de resultados da busca esteja presente
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "gs-webResult.gs-result"))
        )

        # Obter o HTML da página após a busca
        page_source = driver.page_source

        # Parse HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find the first search result element
        first_search_result = soup.find("div", class_="gs-webResult gs-result")

        # Extract the URL from the first search result element
        if first_search_result:
            url_element = first_search_result.find("a", class_="gs-title")
            if url_element:
                result_url = url_element.get("href")
                print(f"URL para '{search_query}': {result_url}")
                urls.append(result_url)
        else:
            print(f"Nenhuma URL encontrada para '{search_query}'")

        # Adicionar um atraso antes da próxima busca
        time.sleep(atraso_entre_buscas)

    # Adicionar a lista de URLs como uma nova coluna no DataFrame, preenchendo com None para buscas sem resultado
    cooperativas_sicredi['URL'] = urls + [None] * (len(cooperativas_sicredi) - len(urls))

    # Salvar o DataFrame em um arquivo CSV
    cooperativas_sicredi.to_csv('df_sicredi_com_url.csv', index=False)

finally:
    # Fechar o navegador
    driver.quit()


URL para 'COOP SICREDI NORTE SC': https://www.sicredi.com.br/coop/norte/
URL para 'COOP SICREDI ARACAJU': https://www.sicredi.com.br/coop/aracaju/
URL para 'COOP SICREDI MEDICRED': https://www.sicredi.com.br/coop/medicred-pr/
URL para 'COOP POL FED - SICREDI POL': https://www.sicredi.com.br/coop/pol/
URL para 'COOP SICREDI CAMPO GRANDE': https://www.sicredi.com.br/coop/campogrande/
URL para 'COOP SICREDI NOROESTE SP': https://www.sicredi.com.br/coop/noroestesp/
URL para 'COOP SICREDI MIL': https://www.sicredi.com.br/coop/mil/
URL para 'SICREDI CREDUNI': https://www.sicredi.com.br/coop/creduni/
URL para 'COOP SICREDI CELEIRO OESTE': https://www.sicredi.com.br/coop/celeiro-co/
URL para 'COOP SICREDI MP': https://www.sicredi.com.br/site/cooperativas/
URL para 'COOP SICREDI AJURIS': https://www.sicredi.com.br/media/produtos/sicredi-ajuris-rs-122018.pdf
URL para 'COOP SICREDI SUL SC': https://www.sicredi.com.br/coop/sulsc/
URL para 'COOP SICREDI VALE SÃO FRANCISCO': https://www.sicredi.com.

In [7]:
# Adicionar uma segunda coluna chamada 'URL_Documentos' com as URLs modificadas
cooperativas_sicredi['URL_Documentos'] = cooperativas_sicredi['URL'].apply(lambda url: url + 'documentos-e-relatorios/' if url else None)

# Salvar o DataFrame em um arquivo CSV
cooperativas_sicredi.to_csv('df_sicredi_com_url.csv', index=False)

In [9]:
import os
# Caminho para o arquivo CSV gerado Manualmente depois dos filtros e limpeza.
caminho_arquivo = 'MANUAL_df_sicredi_com_url.csv'

# Carregar o arquivo CSV em um DataFrame
df = pd.read_csv(caminho_arquivo)

# Iterar sobre as linhas do DataFrame
for indice, linha in df.iterrows():
    nome_instituicao = linha['NOME_INSTITUICAO']
    # Remover caracteres inválidos para nome de pasta
    nome_instituicao = nome_instituicao.replace('/', '_')
    nome_instituicao = nome_instituicao.replace(':', '_')
    nome_instituicao = nome_instituicao.replace('*', '_')
    nome_instituicao = nome_instituicao.replace('?', '_')
    # Criar a pasta se ainda não existir
    if not os.path.exists(nome_instituicao):
        os.makedirs(nome_instituicao)


In [11]:
# Iterar sobre as linhas do DataFrame
for indice, linha in df.iterrows():
    nome_instituicao = linha['NOME_INSTITUICAO']
    url_documentos = linha['URL_Documentos']
    
    if url_documentos and url_documentos != 'none':  # Verificar se a URL não está vazia ou igual a 'none'
        # Remover caracteres inválidos para nome de pasta
        nome_instituicao = nome_instituicao.replace('/', '_')
        nome_instituicao = nome_instituicao.replace(':', '_')
        nome_instituicao = nome_instituicao.replace('*', '_')
        nome_instituicao = nome_instituicao.replace('?', '_')
        
        # Criar a pasta se ainda não existir
        pasta_destino = nome_instituicao
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)
        
        # Fazer scraping da página de documentos
        resposta = requests.get(url_documentos)
        soup = BeautifulSoup(resposta.content, 'html.parser')
        
        # Encontrar todos os links para os documentos
        links_documentos = soup.find_all('a', class_='btn-download-file')
        
        # Iterar sobre os links
        for link in links_documentos:
            url_documento = link['href']
            # Verificar se o documento é de 2022 ou 2023
            if '2022' in url_documento or '2023' in url_documento:
                # Baixar o arquivo
                baixar_arquivo(url_documento, pasta_destino)
